# Meteorological data

Gets AORC data for the domain and time period.

In [ ]:
%matplotlib ipympl

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# setting up logging first or else it gets preempted by another package
import watershed_workflow.ui
watershed_workflow.ui.setup_logging(1)

In [ ]:
import os,sys
import logging
import numpy as np
from matplotlib import pyplot as plt
import pickle
import shapely
import pandas as pd
import geopandas as gpd
import xarray as xr
pd.options.display.max_columns = None

import cftime

import watershed_workflow 
import watershed_workflow.config
import watershed_workflow.sources
import watershed_workflow.mesh
import watershed_workflow.regions
import watershed_workflow.meteorology
import watershed_workflow.io

# set the default figure size for notebooks
plt.rcParams["figure.figsize"] = (8, 6)

## Input: Parameters and other source data

In [ ]:
# Force Watershed Workflow to pull data from this directory rather than a shared data directory.
# This picks up the Coweeta-specific datasets set up here to avoid large file downloads for 
# demonstration purposes.
#
def splitPathFull(path):
    """
    Splits an absolute path into a list of components such that
    os.path.join(*splitPathFull(path)) == path
    """
    parts = []
    while True:
        head, tail = os.path.split(path)
        if head == path:  # root on Unix or drive letter with backslash on Windows (e.g., C:\)
            parts.insert(0, head)
            break
        elif tail == path:  # just a single file or directory
            parts.insert(0, tail)
            break
        else:
            parts.insert(0, tail)
            path = head
    return parts

cwd = splitPathFull(os.getcwd())
assert cwd[-1] == 'workflow'
cwd = cwd[:-1]

# Note, this directory is where downloaded data will be put as well
data_dir = os.path.join(*(cwd + ['input_data',]))
def toInput(filename):
    return os.path.join(data_dir, filename)

output_dir = os.path.join(*(cwd + ['output_data',]))
output_filenames = dict()
def fromOutput(filename):
    return os.path.join(output_dir, filename)    

def toOutput(role, filename):
    output_filenames[role] = filename
    return fromOutput(filename)

# check output and input dirs exist
if not os.path.isdir(data_dir):
    os.makedirs(data_dir, exist_ok=True)
if not os.path.isdir(output_dir):
    os.makedirs(output_dir, exist_ok=True)
       

In [ ]:
# Set the data directory to the local space to get the locally downloaded files
# REMOVE THIS CELL for general use outside fo Coweeta
watershed_workflow.config.setDataDirectory(data_dir)


In [ ]:
## Parameters cell -- this provides all parameters that can be changed via pipelining to generate a new watershed. 
name = 'RussianRiver'
hucs = ['18010110'] # a list of HUCs to run

# Geometric parameters
# -- parameters to clean and reduce the river network prior to meshing
prune_by_area = 10               # km^2
simplify = 125                   # length scale to target average edge 

# -- mesh triangle refinement control
refine_d0 = 200
refine_d1 = 600

refine_L0 = 125
refine_L1 = 300

refine_A0 = refine_L0**2 / 2
refine_A1 = refine_L1**2 / 2

# Refine triangles if they get too acute
min_angle = 20 # degrees

# width of reach by stream order (order:width)
river_widths = dict({1:10, 2:10, 3:20, 4:30, 5:30}) 


# Note that, by default, we tend to work in the DayMet CRS because this allows us to avoid
# reprojecting meteorological forcing datasets.
crs = watershed_workflow.crs.default_crs

# start and stop time for simulation
# note that this is the overlap of AORC and MODIS
start = cftime.DatetimeGregorian(2007, 8, 1)
end = cftime.DatetimeGregorian(2020, 7, 31)

start_noleap = cftime.DatetimeNoLeap(2007, 8, 1)
end_noleap = cftime.DatetimeNoLeap(2020, 7, 31)
cyclic_nyears = 10



In [ ]:
# set up a dictionary of source objects
#
# Data sources, also called managers, deal with downloading and parsing data files from a variety of online APIs.
sources = watershed_workflow.sources.getDefaultSources()

# log the sources that will be used here
watershed_workflow.sources.logSources(sources)


## Reload data

In [ ]:
with open(fromOutput('02_watersheds.pickle'), 'rb') as fid:
    watersheds = pickle.load(fid)

## Download AORC

In [ ]:
# download the data, including:
#  - lazy spatial subsetting to the watershed
#  - chunked temporal_resampling to daily data
#  - streamed directly to file
met_data_request = sources['meteorology'].requestDataset(watersheds.exterior, crs, start, end, temporal_resampling='1D')

time_chunk_size = 100
met_data_chunked = sources['meteorology'].fetchRequest(met_data_request, chunk_time=time_chunk_size)


In [ ]:
# aorc is a lat-lon product -- project it
met_data_projected = watershed_workflow.warp.dataset(met_data_chunked, crs, 'bilinear', time_chunk_size, toOutput('warped meterology', 'warped_meteorology.nc'))


In [ ]:
met_data_projected['time']


In [ ]:
met_data_transient = watershed_workflow.meteorology.convertAORCToATS(met_data_projected)


In [ ]:
# plot a few of the met data -- does it look reasonable?
def plotMetData(met, x=5, y=5):
    """plot one pixel as a function of time"""
    fig = plt.figure()
    ax = fig.add_subplot(221)
    
    met_data_single_pixel = met.isel({'time':slice(0,365*2),
                                               'x' : x,
                                               'y' : y})
    
    met_data_single_pixel['precipitation rain [m s^-1]'].plot(color='b', label='rain')
    met_data_single_pixel['precipitation snow [m SWE s^-1]'].plot(color='c', label='snow')
    ax.set_ylabel('precip [m s^-1]')
    ax.set_title('')
    ax.legend()
    
    ax = fig.add_subplot(222)
    met_data_single_pixel['incoming shortwave radiation [W m^-2]'].plot(color='r', label='qSW_in')
    ax.set_ylabel('incoming shortwave radiation [W m^-2]')
    ax.set_title('')

    ax = fig.add_subplot(223)
    (met['precipitation rain [m s^-1]'][0:365].sum(axis=0)*86400*1000).plot.imshow()
    ax.set_title('annual precip [mm]')

    ax = fig.add_subplot(224)
    met['incoming shortwave radiation [W m^-2]'][0].plot.imshow(cmap='hot')
    ax.set_title('incoming radiation [W m^-2]')

    
    plt.show()

plotMetData(met_data_transient, 50,50)

In [ ]:
met_data_transient['time'][0:3]

In [ ]:
met_data_transient

In [ ]:
# write transient data to disk
filename = toOutput('meteorology_transient', f'{name}_aorc-{start.year}-{end.year}.h5')
watershed_workflow.io.writeDatasetToHDF5(
    filename,
    met_data_transient
    )

In [ ]:
# compute the typical year
met_data_typical = watershed_workflow.meteorology.computeTypicalYear(met_data_transient, cyclic_nyears)


In [ ]:
plotMetData(met_data_typical, 50, 50)

In [ ]:
# write typical data to disk
filename = toOutput('meteorology_typical', f'{name}_aorc_typical-{start.year-cyclic_nyears}-{start.year}.h5')
watershed_workflow.io.writeDatasetToHDF5(
    filename,
    met_data_typical
    )

In [ ]:
# also save an average P - ET = 0.6 * average P
pavg = met_data_transient['precipitation rain [m s^-1]'].mean()

with open(toOutput('total_avg_precip', 'total_avg_precip.pickle'), 'wb') as fid:
    pickle.dump(float(pavg.values), fid)


In [ ]:
# lastly, reread, update, and output filenames
with open(toOutput('05_output_filenames', '05_output_filenames.txt'), 'wb') as fid:
    pickle.dump(output_filenames, fid)